In [2]:
#IMPORTS
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval
import torch
from urllib import request
import random
# device = torch.device("cpu")

In [3]:
# # prepare logger
# logging.basicConfig(level=logging.INFO)

# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# # check gpu
# cuda_available = torch.cuda.is_available()

# print('Cuda available? ',cuda_available)
# if cuda_available:
#   import tensorflow as tf
#   # Get the GPU device name.
#   device_name = tf.test.gpu_device_name()
#   # The device name should look like the following:
#   if device_name == '/device:GPU:0':
#       print('Found GPU at: {}'.format(device_name))
#   else:
#       raise SystemError('GPU device not found')

In [4]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [5]:
from dont_patronize_me import DontPatronizeMe
dpm = DontPatronizeMe('.', '.')
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


In [6]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)
data=dpm.train_task1_df
data

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [7]:

rows = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  item = {
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  }
  rows.append(item)

In [8]:
trdf1 = pd.DataFrame(rows)
trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
8370,8380,refugee,Rescue teams search for survivors on the rubbl...,0
8371,8381,hopeless,The launch of ' Happy Birthday ' took place la...,0
8372,8382,homeless,"The unrest has left at least 20,000 people dea...",0
8373,8383,hopeless,You have to see it from my perspective . I may...,0


In [9]:
test_rows = [] # will contain par_id, label and text
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  test_rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })
  #TODO: keyword?Country?length?

In [10]:
len(rows)

8375

In [11]:
tedf1 = pd.DataFrame(test_rows)
tedf1

,par_id,community,text,label
0,4046,hopeless,We also know that they can benefit by receivin...,1
1,1279,refugee,Pope Francis washed and kissed the feet of Mus...,1
2,8330,refugee,Many refugees do n't want to be resettled anyw...,1
3,4063,in-need,"""Budding chefs , like """" Fred """" , """" Winston ...",1
4,4089,homeless,"""In a 90-degree view of his constituency , one...",1
...,...,...,...,...
2089,10462,homeless,"The sad spectacle , which occurred on Saturday...",0
2090,10463,refugee,""""""" The Pakistani police came to our house and...",0
2091,10464,disabled,"""When Marie O'Donoghue went looking for a spec...",0
2092,10465,women,"""Sri Lankan norms and culture inhibit women fr...",0


In [141]:
tedf1 = tedf1.sample(frac = 1) # shuffle data
# downsample negative instances
pcldf = trdf1[trdf1.label==1]
npos = len(pcldf)

training_set1 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*2]])
# tedf1.par_id
train_data = training_set1
test_data = tedf1
train_data_text_list = train_data.text.to_list()
train_data_label_list = train_data.label.to_list()
test_data_text_list = test_data.text.to_list()
test_data_label_list = test_data.label.to_list()

In [13]:
from sklearn.metrics import accuracy_score, f1_score
def calc_scores_and_print_miscalculations(true_lables, pred_labels, unseen_data, n=10):
    '''prints accuracy, f1 score and first N miscalculations'''
    calc_scores(true_lables, pred_labels)
    # Print predicted labels for unseen test data
    print("\nMisclassifications:")
    c = 0
    for text, true_label, pred_label in zip(unseen_data, true_lables, pred_labels):
        if true_label != pred_label:
            print(f"Pred: {pred_label} Ac: {true_label} Text: {text}")
            c += 1
            if c == n:
                break

def calc_scores(true_lables, pred_labels):
    # Evaluate the model
    accuracy = accuracy_score(true_lables, pred_labels)
    print("Accuracy:", accuracy)

    f1 = f1_score(true_lables, pred_labels)
    print("F1 Score:", f1)
    return (accuracy, f1)

## Benchmark - Bag of words (no augmentation)

In [21]:
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import random

# Download WordNet if not already downloaded
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Joshua\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Joshua\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [100]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

In [175]:
# Synonm augmentation
stop_words = set(stopwords.words('english'))

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def synonym_replacement(text, n=3):
    words = text.split()
    augmented_texts = []
    for _ in range(n):
        new_words = words.copy()
        for i, word in enumerate(words):
            if word.lower() not in stop_words:  # Check if the word is not a stop word
                synonyms = get_synonyms(word)
                if synonyms:
                    synonym = random.choice(synonyms)
                    new_words[i] = synonym
        augmented_texts.append(' '.join(new_words))
    return augmented_texts

# Example usage:
original_text = "Somebody destroyed my only toy"
augmented_texts = synonym_replacement(original_text, n=3)  # Generate 3 augmented texts
for i, augmented_text in enumerate(augmented_texts, 1):
    print(f"Augmented Text {i}: {augmented_text}")
    

def replacement_text(text_list, label_list, transform_f=synonym_replacement):
    aug_text_list = []
    aug_label_list = []
    for text, label in zip(text_list, label_list):
        options = transform_f(text)
        aug_text_list.append(text)
        aug_text_list.extend(options)
        aug_label_list.append(label)
        aug_label_list.extend([label] * len(options))
    return aug_text_list, aug_label_list

from sklearn.cluster import KMeans
from scipy.sparse import vstack
from sklearn.cluster import KMeans
import numpy as np


# Not working fully
def cluster_sampling(X_train_bow, y_train):
    ''' Uses KMEANS and requires that X_train is alreayd in feature form e.g. Bag Of words or TF-IDF, vectorized form'''
    # Step 1: Cluster-based Sampling
    num_clusters = 5  # Number of clusters
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    clusters = kmeans.fit_predict(X_train_bow)

    # Sample size per cluster
    sample_size_per_cluster = 100

    X_train_sampled = []
    y_train_sampled = []

    for cluster_id in range(num_clusters):
        cluster_indices = (clusters == cluster_id).nonzero()[0]
        sampled_indices = np.random.choice(cluster_indices, size=min(sample_size_per_cluster, len(cluster_indices)), replace=False)
        X_train_sampled.extend(X_train_bow[sampled_indices])
        y_train_sampled.extend([y_train[i] for i in sampled_indices])
    return X_train_sampled, y_train_sampled

import random
import numpy as np
from googletrans import Translator
import os
import json
from deep_translator import GoogleTranslator


def translate_group(text_list, label_list):
    aug_text_list = []
    aug_label_list = []
    translator_to = GoogleTranslator(source='en', target=random.choice(['fr']))
    translator_from = GoogleTranslator(source='fr', target='en')
    for text, label in zip(text_list, label_list):
        aug_text_list.append(text)
        new_text = [translator_from.translate(translator_to.translate(text))]
        aug_text_list.extend(new_text)
        
        aug_label_list.append(label)
        aug_label_list.extend([label] * len(new_text))
    return aug_text_list, aug_label_list

import pickle
def write_list_to_file(list1, filename1):
    with open(filename1, 'wb') as file:
        pickle.dump(list1, file)
def read_list_from_file(filename1):
    with open(filename1, 'rb') as file:
        return pickle.load(file)
    
res1, res2 = translate_group(train_data_text_list , train_data_label_list)
write_list_to_file(res1, "french_translation_text.txt")
write_list_to_file(res2, "french_translation_label.txt")

Augmented Text 1: person put_down my only miniature
Augmented Text 2: somebody destruct my only diddle
Augmented Text 3: someone destruct my only play


In [176]:
# Computationally expensive transformations
X_aug, y_aug = replacement_text(train_data_text_list, train_data_label_list, synonym_replacement)

In [191]:
# X,y = train_data_text_list, train_data_label_list
X,y = read_list_from_file("french_translation_text.txt"), read_list_from_file("french_translation_label.txt")

In [187]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from scipy.sparse import vstack
# Example dataset (X: text data, y: sentiment labels)
print(len(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define stopwords and create CountVectorizer with n-grams and stopwords removal
stop_words = list(stop_words)
vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=stop_words)

# Create BoW vectors
maxv = -float('inf')
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Step 2: Train a logistic regression classifier on the sampled data
classifier = LogisticRegression()
classifier.fit(X_train_bow, y_train)

# Step 3: Evaluate the model
y_pred = classifier.predict(X_test_bow)
calc_scores(y_test, y_pred)
# Unseen data
unseen_data = test_data_text_list
true_labels = test_data_label_list

# Vectorize the unseen test data using the same CountVectorizer instance
X_unseen_bow = vectorizer.transform(unseen_data)

# Predict sentiment labels for the unseen test data
y_unseen_pred = classifier.predict(X_unseen_bow)

# Calculate accuracy
acc, f1 = calc_scores(true_labels, y_unseen_pred)
        
# calc_scores_and_print_miscalculations(true_labels, y_unseen_pred, unseen_data)
print(len(X_aug))

1200
Accuracy: 0.95
F1 Score: 0.9508196721311475
Accuracy: 0.7125119388729704
F1 Score: 0.3127853881278539
9528


## Benchmark model - SVM - TF-IDF feature representation

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [192]:

# Split the dataset into training and testing sets
# X,y = train_data_text_list[:300] + train_data_text_list[-300:], train_data_label_list[:300] +  train_data_label_list[-300:] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train an SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_tfidf, y_train)

# Predict sentiment labels for the test set
y_pred = svm_classifier.predict(X_test_tfidf)

calc_scores(y_test, y_pred)

# Example of unseen test data
unseen_data = test_data_text_list
true_labels = test_data_label_list # True sentiment labels for the unseen test data

# Convert the unseen test data into TF-IDF vectors
X_unseen_tfidf = tfidf_vectorizer.transform(unseen_data)

# Predict sentiment labels for the unseen test data
y_unseen_pred = svm_classifier.predict(X_unseen_tfidf)

# Calculate accuracy for unseen test data
calc_scores_and_print_miscalculations(true_labels, y_unseen_pred, unseen_data)



Accuracy: 0.9166666666666666
F1 Score: 0.9173553719008265
Accuracy: 0.6575931232091691
F1 Score: 0.2865671641791045

Misclassifications:
Pred: 1 Ac: 0 Text: Tottenham are still homeless . The club expected to be in their new stadium for their match against Liverpool last Saturday but testing threw up safety issues .
Pred: 1 Ac: 0 Text: Children in Ky feel the effects of homelessness <h> HOMELESSNESS is threatening children 's education , a new report has found .
Pred: 1 Ac: 0 Text: It 's incumbent upon Kenyans not to fall prey to consuming political gimmickry , especially ahead of the election year . It is imponderable that a government can launch projects that are n't budgeted for , but still insist it is n't faking them . Ruto knows he can sell a fad such as name-calling NASA without anticipating calls to substantiate . But I also think politicians around the presidency are increasingly vulnerable to being fed all manner of social media chit-chat as facts .
Pred: 1 Ac: 0 Text: In oth